In [1]:
import glob
import matplotlib.pyplot as plt
from elecpy.elecpySession import ElecpySession
import pandas as pd
import copy
import time
import os, sys
import shutil
import json
import numpy as np

import chainer
from chainer import cuda

from matplotlib import animation, rc
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import elecpy.elecpy as elp
from elecpy.util.cmap_bipolar import bipolar
from elecpy.elecpySession import ElecpySession

from IPython.display import HTML

from opmap.videoData import VideoData
from opmap.vmemMap import VmemMap
from opmap.phaseMap import PhaseMap
from opmap.phaseVarianceMap import PhaseVarianceMap

from scipy.signal import argrelmax
from numba.decorators import autojit

%matplotlib inline


In [2]:
src_dir = '/mnt/Omer/Project/01.PhaseDiscontinuity/SimulationResults/'
save_dir = '/mnt/Omer/Project/01.PhaseDiscontinuity/AnalysisResults/2019-04-05/'
print(save_dir)
if not os.path.exists(save_dir) : os.mkdir(save_dir)

/mnt/Omer/Project/01.PhaseDiscontinuity/AnalysisResults/2019-04-05/


In [3]:
df = pd.DataFrame( {
    'setting' : ['original', '(1)', '(2)', '(3)'] ,
    'session_id' : ['20190331-1','20190331-2','20190331-3','20190331-4'],
    'case_id' : ['Mahajan', 'A', 'B', 'C, D'],
    'simu_id' : [None, '20190226-3', '20180616-1', '20180625-1']
    
})
df

,case_id,session_id,setting,simu_id
0,Mahajan,20190331-1,original,None
1,A,20190331-2,(1),20190226-3
2,B,20190331-3,(2),20180616-1
3,"C, D",20190331-4,(3),20180625-1


## Mahajan

In [4]:
!cat ./elecpy/cell/mahajan/kernel.c | grep gca_
!cat ./elecpy/cell/mahajan/kernel.c | grep gtof_
!cat ./elecpy/cell/mahajan/kernel.c | grep gtos_
!cat ./elecpy/cell/mahajan/kernel.c | grep " _dh "
!cat ./elecpy/cell/mahajan/kernel.c | grep " _dj "

!cat ./elecpy/cell/mahajan/const.py | grep gna_
!cat ./elecpy/cell/mahajan/const.py | grep gkr_
!cat ./elecpy/cell/mahajan/const.py | grep gks_

double _gca = {gca_} * pow({Q10CAL_}, (temp-{temp_})/10.0)/1.4;
double _gtof = {gtof_} * pow({Q10TO_}, (temp-{temp_})/10.0)*1.4;
double _gtos = {gtos_} * pow({Q10TO_}, (temp-{temp_})/10.0)*1.4;
double _dh = (_ah*(1.00000-h) - _bh*h)*1.4;
_h = h + _dh * dt;
double _dj = (_aj*(1.00000-j) - _bj*j)*1.4;
_j = j + _dj * dt;
  'gna_'         : 12.0              , #
  'gkr_'         : 0.0125            , #
  'gks_'         : 0.1386            , #


# シミュレーション実行

In [5]:
# load sample parameter
with open ('sim_params_org.json','r') as f:
    sim_params = json.load(f)

# modify parameters
sim_params['time']['end'] = 200
sim_params["stimulation"] = {
    "extracellular": [
        {
            "start": 10.0, 
            "shape": [
                200, 
                200
            ], 
            "train": 1, 
            "name": "edge_L", 
            "amplitude": -150.0, 
            "duration": 20.0, 
            "interval": 200.0, 
            "size": 10
        }
    ], 
    "membrane": []
}
sim_params['log']['path'] = '/mnt/Omer/Project/01.PhaseDiscontinuity/SimulationResults/20190405-1/'

sim_params['geometory']['height'] = 200
sim_params['geometory']['width'] = 200

del(sim_params["restart"])

#sim_params['restart'] = {}
#sim_params['restart']['count'] = 0
#sim_params['restart']['source'] = "/mnt/recordings/SimulationResult/20171128-1/"
#sim_params['restart']['source'] = "/mnt/Omer/Project/01.PhaseDiscontinuity/SimulationResults/20171128-1/"

print json.dumps(sim_params, indent=4)

{
    "cell_type": "mahajan", 
    "log": {
        "path": "/mnt/Omer/Project/01.PhaseDiscontinuity/SimulationResults/20190405-1/", 
        "cnt": 1000
    }, 
    "stimulation": {
        "extracellular": [
            {
                "start": 10.0, 
                "shape": [
                    200, 
                    200
                ], 
                "train": 1, 
                "name": "edge_L", 
                "amplitude": -150.0, 
                "duration": 20.0, 
                "interval": 200.0, 
                "size": 10
            }
        ], 
        "membrane": []
    }, 
    "time": {
        "end": 200, 
        "udt": 0.001
    }, 
    "geometory": {
        "width": 200, 
        "ds": 0.015, 
        "height": 200
    }
}


In [ ]:
g = elp.sim_generator(sim_params)

path_dst = sim_params["log"]["path"]
if not os.path.exists( path_dst ): os.mkdir( path_dst )
with open(os.path.join(path_dst, 'sim_params.json'),'w') as f: json.dump(sim_params, f, indent=4)

out = []
while True:
    img = g.next()
    if img is False: break
    out.append(np.copy(img))

elecpy simulation start!
Stimulation settings ...done
Allocating data... ...done
Initializing data... ...done
Building PDE system ... ...done
Main loop start!
------------------0.0/200ms

In [ ]:
fig = plt.figure()
plt.axis('off')

ims = []
for img in out:
    im = plt.imshow(
        img.reshape(200,200),
        vmin = -100.0, vmax = 20.0,
        cmap=bipolar(neutral=0, lutsize=1024),
        interpolation='nearest')
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims)
#ani.save('anim.gif', writer="imagemagick")
#ani.save('anim.mp4', writer="ffmpeg")
#plt.show()

HTML(ani.to_html5_video())